# Detección de frescura en frutas

En la solución del problema se emplean tres modelos de aprendizaje profundo basados en redes convolucionales. El modelo simple que mejor se desempeña es la tarea constituye un modelo ResNet con 5 bloques residuales y 20 capas referido como ResNet20 con un 99.0% de accuracy. Se realiza un ensemble con los modelos entrenados, alcanzando un accuracy del 99.3%.

**Estructura del proyecto:**

1. Análisis exploratorio de datos.
2. Preprocesamiento de los datos.
3. Selección de modelos.
4. Entrenamiento.
5. Evaluación del entrenamiento.
6. Evaluación del modelo.


## Análisis exploratorio de datos

Los datos se dividen originalmente en dos conjuntos, uno de entrenamiento (23619) y otro de pruebas (6738). En estos conjuntos se encuentran representadas 18 clases pertenecientes a 9 frutas cada clasificada además en frescas o podridas.

![Balance todas las clases](metrics/all_classes_dataset_info.png)

En la gráfica anterior se observa:
- Desbalance entre la cantidad de elementos entre los distintos tipos de frutas
- En una misma clase de fruta la cantidad de elementos frescos y podridos del conjunto se encuentran en proporciones similares
- Algunas clases de frutas no se encuentran representadas en el conjunto de pruebas.

<div align="center">
   <img src="metrics/binary_dataset_info.png" alt="Balance frescos-podridos"/>
</div>

El alto y ancho de las fotos, así como su proporción son heterogéneos:

<table>
   <tr>
      <td> <img src="metrics/size_dataset_info.png" alt="Drawing"/> </td>
      <td> <img src="metrics/size_ratio_dataset_info.png" alt="Drawing"/> </td>
   </tr>
</table>

## Preprocesamiento de datos

Los datos se dividieron en tres conjuntos:

- Entrenamiento: 18896 elementos (representa el 80% de las muestras encontradas en la sección de entrenamiento original).
- Validación: 4723 elementos (representa el 20% de las muestras encontradas en la sección de entrenamiento original).
- Prueba: 6738 elementos.

### Procesamiento y aumento de datos

Para aumentar la variedad de los datos y hacer más robustos los modelos se emplearon un conjunto de transformaciones que se aplicaron sobre los elementos de entrenamiento:

- Aumentar el tamaño de la imagen y tomar porciones aleatorias de 256 píxeles por 256 píxeles.
- Voltear aleatoriamente la imagen de forma horizontal o vertical.
- Rotar aleatoriamente la imagen entre 0 y 180 grados.
- Hacer zoom aleatoriamente a la imagen.
- Añadir o quitar aleatoriamente brillo a la imagen.

<table>
   <th>Original</th>
   <th>Aumentado</th>
   <tr>
      <td> <img src="metrics/original_data_augmentation.png" alt="Drawing"/> </td>
      <td> <img src="metrics/data_augmentation.png" alt="Drawing"/> </td>
   </tr>
</table>

## Selección de modelos

Para el trabajo con imágenes es usado comúnmente las CNN o redes convolucionales. Este tipo de modelo contiene hipótesis generalmente ciertas para las imágenes tales como:

- Los patrones aprendidos son invariante a traslaciones.
- Pueden aprender herarquías de patrones espaciales.

Para la solución del problema se utilizan tres arquitecturas de redes convolucionales:

- ResNet20: 
  - Entrenada desde 0.
  - 12,302,802 parámetros
- ResNet50V2
  - Preentrenada con ImageNet
  - 23,601,682 parámetros
- Xception
  - Preentrenada con ImageNet
  - 20,898,362 parámetros

Los modelos ResNet50V2 y Xception fueron seleccionados de acuerdo a su tamaño y métricas en comparación con los otros existentes con un balance tamaño-precisión buenos para los recursos utilizados.

## Entrenamiento

Para el entrenamiento se decide utilizar las mismas clases originales, sin agrupar las etiquetas a frescas o podridas, de esta forma el modelo se vuelve más general aún permitiendo la clasificación deseada.

**Métricas**

Las métricas calculadas en el entrenamiento corresponden a las versiones multiclases de cada una de estas. Las métricas correspondientes a la clasificación binaria son calculadas posteriormente. Las métricas son:

- Accuracy
- F1-Score
- Precision
- Recall
- AUC-ROC

**Optimizador**

- Adam

**Función de pérdida**

- Categorical crossentropy

### ResNet20

**Callbacks:**

- Multiplicar la tasa de aprendizaje por 0.1 luego de cierto tiempo que no se encuentre una mejora de la función de pérdida en el conjunto de validación.
- El entrenamiento se detiene si no se encuentra una mejora en la función de pérdida en el conjunto de validación en 20 épocas.
- Se salva solo el modelo con mejor valor en la función de périda del conjunto de validación.

**Valores iniciales:**

- Tasa de aprendizaje: 0.1 (Alto, disminuye con el tiempo)
- Batch size: 32
- Máximo de épocas: 200

### Transfer Learning

De los modelos preentrenados (ResNet50V2 y Xception) solamente se utiliza la parte de extracción de atributos de las imágenes, eliminando su sección utilizada para clasificación. Para clasificar las clases se agregan capas a la salida del modelo:

- GlobalAveragePooling2D: Condensa los filtros en una sola dimensión.
- Dropout: Combate el overfitting del modelo a las clases.
- Dense: Actúa como clasificador final.

El entrenamiento en este tipo de modelos se divide en dos secciones:

1. Ajuste de los pesos de la nueva capa densa agregada: Dado que la nueva capa esta inicializada con pesos aleatorios si se entrenara con esta así, el gradiente destruiría los pesos aprendidos por el modelo.
    1. Se congela todo el modelo excepto las capas agregadas 
    2. Se entrena la nueva capa con el conjunto de datos con una tasa de aprendizaje mediana.
2. Ajuste fino del modelo con algunas capas descongeladas: Una vez la capa densa está ajustada al modelo y a los datos el gradiente es menor, propiciando un mejor aprovechameinto de los pesos previamente aprendidos por el modelo.
    1. Se descongela parte del modelo cercano al clasificador. Esta sección del modelo tiende a aprender características más específicas del problema, por lo que conviene reentrenarlas para el nuevo dominio en que se trabaja.
    2. Se entrena el modelo con el conjunto de datos con una tasa de aprendizaje baja.

**Callbacks preentrenamiento:**

- El entrenamiento se detiene si no se encuentra una mejora en la función de pérdida en el conjunto de validación en 5 épocas.

**Valores iniciales preentrenamiento:**

- Tasa de aprendizaje: 0.001
- Batch size: 32
- Máximo de épocas: 50

**Callbacks fine tuning:**

- El entrenamiento se detiene si no se encuentra una mejora en la función de pérdida en el conjunto de validación en 10 épocas.
- Se salva solo el modelo con mejor valor en la función de périda del conjunto de validación.

**Valores fine tuning:**

- Tasa de aprendizaje: 0.00001
- Batch size: 32
- Máximo de épocas: 200


## Evaluación del entrenamiento

### ResNet20

En la siguiente curva de aprendizaje se evidencia un comportamiento con errático en el conjunto de validación hasta la época 20, esto se debe probablemente al alto valor en la tasa de aprendizaje que en la época 20 disminuye. Una vez este valor decrece la pérdida en el conjunto de validación se estabiliza y deja de mejorar. El entrenamiento termina en la época 41, lo que implica que el modelo a utilizar es el entrenado en la época 21.

![ResNet20 loss](model/train_resnet20.Epochs_loss_val_loss.png)

Similar a la curva de aprendizaje anterior el accuracy del modelo se estabiliza una vez decrece la tasa de aprendizaje

![ResNet20 acc](model/train_resnet20.Epochs_categorical_accuracy_val_categorical_accuracy.png)

Las curvas de aprendizaje no muestran señales de overfitting a los datos y se considera que el proceso de aprendizaje fue satisfactorio.

### ResNet50V2

La curva de aprendizaje se divide en dos momentos, el de preentrenamiento y el de fine tuning. La transición puede verse claramente cuando la pérdida aumenta su descenso a mitad del proceso. El proceso de entrenamiento se termina en la época 68, indicando que el mejor modelo fue el de la época 58.

![ResNet50 loss](model/train_resnet50.Epochs_loss_val_loss.png)

El cambio de preentrenamiento a fine tuning también se ve reflejado en la curva de accuracy de manera similar.

![ResNet50 acc](model/train_resnet50.Epochs_categorical_accuracy_val_categorical_accuracy.png)

Las curvas de aprendizaje no muestran señales de overfitting. Los valores obtenidos por este modelo son inferiores a los alcanzados por ResNet20.

### Xception

La curva de aprendizaje se divide en dos momentos, el de preentrenamiento y el de fine tuning. Al igual que en ResNet50V2, la transición puede verse claramente cuando la pérdida aumenta su descenso a mitad del proceso. El proceso de entrenamiento se termina en la época 87, indicando que el mejor modelo fue el de la época 77.

![Xception loss](model/train_xception.Epochs_loss_val_loss.png)

El cambio de preentrenamiento a fine tuning también se ve reflejado en la curva de accuracy de manera similar.

![Xception acc](model/train_xception.Epochs_categorical_accuracy_val_categorical_accuracy.png)

Las curvas de aprendizaje no muestran señales de overfitting. Los valores obtenidos por este modelo son inferiores a los alcanzados por ResNet20 pero superiores a ResNet50V2.

## Evaluación del modelo

### ResNet20

**Matriz de confusión de todas las clases:**

Se muestra que el modelo se equivoca más en la clasificación de papas podridas, las cuales las clasifica como papas frescas (30), luego se encuentra la clasificación incorrecta de pepinos frescos por quimbombó fresco (17).

![All Confusion Matrix Resnet20](metrics/resnet20/all_classes_confusion_matrix.png)

**Matriz de confusión binaria:**

Se observa una tendencia mayor a clasificar en frescos alimentos podridos.

![Binary Confusion Matrix Resnet20](metrics/resnet20/binary_confusion_matrix.png)

**Curva ROC:**

Presenta una curva en la el radio de verdaderos positivos es casi en su totalidad 1.

![ROC Resnet20](metrics/resnet20/roc.png)

### ResNet50V2

**Matriz de confusión de todas las clases:**

Se muestra que el modelo se equivoca más en la clasificación de papas frescas, las cuales las clasifica como papas podridas (22), a diferencia de ResNet20 este modelo presenta más fallos en sus clasificaciones y son más dispersas.

![All Confusion Matrix Resnet50](metrics/resnet50/all_classes_confusion_matrix.png)

**Matriz de confusión binaria:**

Se observa una tendencia mayor a clasificar en podridos alimentos frescos, un comportamiento inverso al de ResNet20.

![Binary Confusion Matrix Resnet50](metrics/resnet50/binary_confusion_matrix.png)

**Curva ROC:**

Presenta una curva por encima de la curva aleatoria, con un rendimiento inferior al de ResNet20.

![ROC Resnet50](metrics/resnet50/roc.png)

### Xception

**Matriz de confusión de todas las clases:**

Se muestra que el modelo, al igual que ResNet20, se equivoca más en la clasificación de papas podridas, las cuales las clasifica como papas frescas (21), luego se encuentra la clasificación incorrecta de quimbombó frescos por quimbombó podrido (16).

![All Confusion Matrix Xception](metrics/xception/all_classes_confusion_matrix.png)

**Matriz de confusión binaria:**

Se observa una tendencia mayor a clasificar en frescos alimentos podridos, al igual que ResNet20.

![Binary Confusion Matrix Xception](metrics/xception/binary_confusion_matrix.png)

**Curva ROC:**

Presenta una curva más pronunciada para arriba que ResNet50V2, indicando una clasificación más robusta.

![ROC Xception](metrics/xception/roc.png)

### Ensemble

Se hizo un ensemble por conteo con los tres modelos entrenados, en caso de empate se selecciona la clase inferida por el modelo ResNet20. Este modelo se desempeña mejor de manera general que sus elementos individuales.

**Matriz de confusión de todas las clases:**

La mayor afectación se observa en la clasificación incorrecta por el modelo de varias frutas en distintos estados como manzana fresca, esto ocurre unas 36 veces. Se observa también que el error al calsificar papas disminuye en comparación con los demás modelos.

![All Confusion Matrix Ensemble](metrics/ensemble/all_classes_confusion_matrix.png)

**Matriz de confusión binaria:**

Se observa una tendencia mayor a clasificar en frescos alimentos podridos, esto es probablemente a que la mayoría de los modelos (ResNet20 y Xception) poseen este comportamiento.

![Binary Confusion Matrix Ensemble](metrics/ensemble/binary_confusion_matrix.png)

**Curva ROC:**

Dado que el método de ensemble utilizado no utiliza de manera directa las probabilidades, estas fueron aproximadas como el promedio de las distribuciones de los tres modelos. Notar que la clase resultante del mayor promedio no tiene por que ser la seleccionada por el método de conteo.

![ROC Ensemble](metrics/ensemble/roc.png)

### Métricas comparativas

| Modelo   | Accuracy | Precision | Recall  | F1      |Binary Accuracy| Binary Precision | Binary Recal | Binary F1 | AUC-ROC |
| --       | --       | --        | --      | --      | --            | --               | --           | --        | --      |
|ResNet20  | 0.983    | 0.976     | 0.975   | 0.975   | 0.990         | 0.990            | 0.990        | 0.990     | **1.00**|
|ResNet50V2| 0.965    | 0.889*    | 0.889*  | 0.888*  | 0.979         | 0.980            | 0.978        | 0.979     | 0.765   |
|Xception  | 0.983    | 0.973     | 0.974   | 0.973   | 0.987         | 0.987            | 0.988        | 0.987     | 0.813   |
|Ensemble  | **0.990**| **0.989** |**0.985**|**0.987**| **0.993**     | **0.993**        | **0.993**    | **0.993** | 0.822** |

- *: Esta métrica toma en cuenta una clase con ninguna participación en el conjunto de entrenamiento, por lo que sus valores de precisión y recobrado son 0, contribuyendo negativamente a estas métricas.
- **: Los porcientos son calculados como el promedio devuelto por los modelos, la clase resultante del mayor promedio no tiene por que ser la seleccionada por el método de conteo.

### Resumen

- Se observa como el método ensemble logra un mejor resultado en la mayoría de las métricas calculadas.
